In [1]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [2]:
# Create engine using the `spacex.sqlite` database file
engine = create_engine("sqlite:///tornadoes_clean.sqlite")

In [3]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

tornadoes
-----------
id BIGINT
tornado_id BIGINT
yr BIGINT
month BIGINT
date TEXT
state TEXT
category FLOAT
injuries BIGINT
fatalities BIGINT
loss FLOAT
start_lat FLOAT
start_longitude FLOAT
end_latitude FLOAT
end_longitude FLOAT
distance_traveled FLOAT
width BIGINT
seasons TEXT



In [4]:
# user inputs
user_year = 2022
where_clause = f"yr = {user_year}"

# switch on user_year
if user_year != 'All':
    where_clause = f"yr = {user_year}"
else:
    where_clause = f"yr > 2000"

# build the query
query = f"""
    SELECT
        yr,
        month,
        date,
        state,
        category,
        injuries,
        fatalities,
        loss,
        start_lat,
        start_longitude,
        end_latitude,
        end_longitude,
        distance_traveled,
        width,
        seasons
    FROM
        tornadoes
    WHERE
        {where_clause}
    ORDER BY
        date DESC;
"""

# execute query
df = pd.read_sql(text(query), con=engine.connect())
df.head()

,yr,month,date,state,category,injuries,fatalities,loss,start_lat,start_longitude,end_latitude,end_longitude,distance_traveled,width,seasons
0,2022,12,2022-12-30,MS,1.0,0,0,52000.0,31.3800,-90.5500,31.3900,-90.4100,8.29,100,winter
1,2022,12,2022-12-29,TX,1.0,0,0,15000.0,30.5840,-94.2611,30.5900,-94.2600,0.42,75,winter
2,2022,12,2022-12-15,GA,2.0,0,0,1000000.0,31.3039,-83.9463,31.3290,-83.9309,1.96,280,winter
3,2022,12,2022-12-15,LA,1.0,0,0,20000.0,30.1120,-91.6390,30.1130,-91.6380,0.09,50,winter
4,2022,12,2022-12-15,FL,1.0,0,0,100000.0,28.4310,-82.4729,28.4681,-82.3088,10.30,100,winter


In [6]:
# user inputs
user_year = "All"
where_clause = f"yr = {user_year}"

# switch on user_year
if user_year != 'All':
    where_clause = f"yr = {user_year}"
else:
    where_clause = f"yr > 2000"

# build the query
query = f"""
    SELECT
        yr,
        state,
        category,
        loss,
        start_lat,
        start_longitude,
        end_latitude,
        end_longitude,
        distance_traveled,
        width
    FROM
        tornadoes
    WHERE
        {where_clause}
    ORDER BY
        date DESC;
"""

# execute query
df = pd.read_sql(text(query), con=engine.connect())
df.head(50)

,yr,state,category,loss,start_lat,start_longitude,end_latitude,end_longitude,distance_traveled,width
0,2022,MS,1.0,52000.0,31.3800,-90.5500,31.3900,-90.4100,8.29,100
1,2022,TX,1.0,15000.0,30.5840,-94.2611,30.5900,-94.2600,0.42,75
2,2022,GA,2.0,1000000.0,31.3039,-83.9463,31.3290,-83.9309,1.96,280
3,2022,LA,1.0,20000.0,30.1120,-91.6390,30.1130,-91.6380,0.09,50
4,2022,FL,1.0,100000.0,28.4310,-82.4729,28.4681,-82.3088,10.30,100
5,2022,FL,1.0,500000.0,27.7744,-82.7345,27.7820,-82.7050,1.88,100
6,2022,FL,0.0,0.0,27.0700,-82.2200,27.0724,-82.2132,0.14,100
7,2022,MS,0.0,0.0,33.1200,-90.1900,33.1900,-90.1400,5.91,100
8,2022,MS,0.0,0.0,32.0800,-90.3200,32.0900,-90.3100,0.80,70
9,2022,LA,0.0,0.0,30.7663,-92.2172,30.7663,-92.2172,0.01,1


In [ ]:
# user inputs
user_seasons = "All"

# build the query
if user_seasons != 'All':
    where_clause = "seasons = :user_seasons"
else:
    where_clause = "1 = 1"  # This will always be true and effectively ignore the WHERE clause

query = f"""
    SELECT
        yr,
        month,
        state,
        category,
        injuries,
        fatalities,
        seasons
    FROM
        tornadoes
    WHERE
        {where_clause}
    ORDER BY
        date DESC;
"""

# execute query
with engine.connect() as connection:
    df_seasons = pd.read_sql(query, connection, params={'user_seasons': user_seasons} if user_seasons != 'All' else {})
    print(df_seasons.tail())